In [3]:
import allel
import zarr
import numpy as np
#import malariagen_data
import pandas as pd
import plotly.express as px
from tqdm.notebook import tqdm
import sgkit as sg
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SeqIO

# Set plot style
sns.set(style="whitegrid")


In [4]:
# Define metadata and qc bool globally to start
#define useful variables
zarr_base_path = f"/scratch/user/uqtdenni/afar_production_bunya/curation/uq-beebe-001/combined_zarr/{{contig}}.zarr"

# Let's start by converting zarrs for the 5 largest contigs - we can do QC on them...
ref_path = '/scratch/user/uqtdenni/afar_production_bunya/reference/VectorBase-54_AfarautiFAR1_Genome.fasta'
# now let's get a list of the contigs that we are going to call over
contig_lengths = {}
for record in SeqIO.parse(ref_path, "fasta"):
    seq_id = record.id
    seq_length = len(record.seq)
    contig_lengths[seq_id] = seq_length
filtered_contigs = {k: v for k, v in sorted(contig_lengths.items(), key=lambda item: item[1], reverse=True) if v > 100000}

# Because these data are unstaged, we need to faff about a bit more and load the unstaged metadata to exclude extra dud samples
df_samples_dirty = pd.read_csv('/scratch/user/uqtdenni/far_hin_1.x/work/metadata_development_20250702/sample_metadata_interim_seq_qc_pass.txt', index_col = 'derived_sample_id')
# And load the final (cleaned) metadata
df_samples = pd.read_table('/scratch/user/uqtdenni/far_hin_1.x/work/metadata_development_20250702/sample_metadata_pass_qc_fix.txt', index_col = 'derived_sample_id')

# Mask removing samples we removed before the staging step of QC (that I haven't done yet)
qc_bool = df_samples_dirty.index.isin(df_samples.index)

# Zarr location
zarr_base_path = f"/scratch/user/uqtdenni/afar_production_bunya/curation/uq-beebe-001/combined_zarr/{{contig}}.zarr"


In [23]:
# Define helper functions
def load_genotype_array(contig, qc_bool=qc_bool, df_samples=df_samples, sample_query = None, n_snps=None, is_segregating=None):
    # Load gts and remove failed qc samples
    z = zarr.open(zarr_base_path.format(contig=contig))
    
    # Variant-level mask: punctulatus_group_filter_pass
    filter_mask = z[f"{contig}/punctulatus_group_filter_pass"][:]
    
    # Apply combined variant mask
    gt = allel.GenotypeChunkedArray(z[f"{contig}/calldata/GT"])
    gt = gt.compress(qc_bool, axis=1)          # Filter samples by QC
    gt = gt.compress(filter_mask, axis=0)    # Filter variants
    
    # If an additional mask is supplied to subset the data from the finished metadata, apply, else return all samples
    if sample_query is not None:
        bool_query = np.array(df_samples.eval(sample_query))
        gt = gt.compress(bool_query, axis=1)

    if is_segregating is not None:
            is_seg = gt.count_alleles()[:].is_segregating()
            gt = gt.compress(is_seg)

    if n_snps is not None:
            gt = select_random_genotypes_sorted(gt, n_snps)

    return gt

def load_position(contig):
        z = zarr.open(zarr_base_path.format(contig = contig))
        filter_mask = z[f"{contig}/punctulatus_group_filter_pass"][:]
        pos =  np.array((z[f"{contig}/variants/POS"]))
        return pos.compress(filter_mask)

def pc1_along_genotype_array(contig, window_size,n_snps_per_window=None, sample_set=None, sample_query=None, metadata=df_samples, colour=None, title=None):
    """
    Calculates PC1 along a chrom, using a stepping window of size `window_size`. Either a chrom 
    ('2L', '3RL', etc) and a sample_set can be passed, 
    or a tuple of genotype array and position array can be passed.
    
    Parameters
    ----------
    chrom: str or tuple
        Either a string contig ('2L', '3RL', etc) and sample_set/sample_query can be passed which will invoke
        malariagen_data, or a list of both a genotype array and an array of genomic positions.
    window_size: int
        Size of the sliding window in snps.
    n_snps_per_window: int
        Number of snps to use in each window. If None, all snps are used.
    sample_set: str
        Sample set to use. If None, all samples are used.
    sample_query: str
        Sample query to use. If None, all samples in the sample set are used.
    metadata: pandas dataframe
        Metadata for the samples. Must be same length and order as genotype array. Columns can be used for plotting colours
    colour: str
        Column name in metadata to use for colouring the plot.
    """

    # Subset metadata if we want to
    if sample_query:
        metadata = metadata.query(sample_query)
    else:
        metadata = metadata
    
    # Load and filter genotypes
    g = load_genotype_array(contig=contig, qc_bool = qc_bool, sample_query=sample_query) # Load genotypes
    ac = g.count_alleles() #Count alleles
    flt = (ac.max_allele() == 1) & (ac[:, :2].min(axis=1) > 1) # Remove singletons and keep only biiallelics
    gf = g.compress(flt, axis=0) #Apply filter
    gn_alt = gf.to_n_alt()
    
    #print("removing any invariant sites")
    #loc_var = np.any(gn_alt != gn_alt[:, 0, np.newaxis], axis=1)
    #gn_var = np.compress(loc_var, gn_alt, axis=0)

    #Load pos
    pos = load_position(contig = contig)
    
    print("computing pcs")
    pc1s = allel.moving_statistic(values=gn_alt, statistic=pca, size=window_size, n_snps_per_window=n_snps_per_window)
    midpoints = allel.moving_statistic(values=pos[flt], statistic=np.median, size=window_size)

    # Align to first window so all are consistent
    pc1s = align_pc1s_across_windows(pc1s)

    pc_df = pd.DataFrame(pc1s).T
    pc_df.columns = midpoints   
    pc_df = pd.concat([metadata.reset_index(), pc_df.reset_index(drop=True)], axis=1)
    pc_df = pc_df.melt(id_vars=metadata.columns, var_name='midpoint', value_name='PC1')
    pc_df['PC1_scaled'] = pc_df.groupby('midpoint', group_keys=False)['PC1'].apply(lambda x: (x-np.mean(x))/(max(x)-min(x)))

    # Set plot style
    #sns.set(style="ticks")

    # Create the plot
    #plt.figure(figsize=(17, 5))
    #sns.lineplot(data=pc_df, 
    #            x="midpoint", 
    #            y="PC1_scaled", 
    #            hue=colour, 
    #            units="sample_id",  # Ensures each individual gets its own line
    #            estimator=None,      # Prevents aggregation
    #            linewidth=0.5,       # Thinner lines
    #            palette=pop_code_cols, # Apply custom colors
    #            alpha=0.5)           # Transparency

    #sns.despine()
    # Improve legend placement
    #plt.legend(title="Cohort", bbox_to_anchor=(1.05, 1), loc='upper left')

    # Show plot
    #plt.xlabel("Position")
    #plt.ylabel("PC1_scaled")
    #plt.title(title)
    #plt.savefig(f'/Users/dennistpw/Projects/AsGARD/figures/{title}.svg', format='svg')
    #plt.show()

    return(pc_df)

def pca(x, n_components=1, n_snps_per_window=None):

    """
    Performs PCA and returns the first principal component.
    """
    # optionally thin snps
    if n_snps_per_window:
        i_mask = np.random.choice(x.shape[0], n_snps_per_window)
        x = x.take(i_mask, axis=0)

    coords, _ = allel.pca(x, n_components=n_components)
    
    # flip axes back so PC1 is same orientation in each window 
    for i in range(n_components):
        c = coords[:, i]
    if np.abs(c.min()) > np.abs(c.max()):
        coords[:, i] = c * -1
                
    return(coords[:, 0])
    
def align_pc1s_across_windows(pc1s):
    """
    Ensures consistent PC1 orientation across windows by aligning to the first window.
    """
    reference_pc1 = pc1s[0]  # Use the first window as reference

    for i in range(1, len(pc1s)):
        if np.corrcoef(reference_pc1, pc1s[i])[0, 1] < 0:
            pc1s[i] *= -1  # Flip sign if correlation is negative

    return pc1s
    

In [24]:
pc_test = pc1_along_genotype_array(contig = "KI915040", window_size = 50_000)

computing pcs


TypeError: Could not convert string 'far8_98_89_10far8_98_89_18far8_98_89_25far8_98_89_26far8_98_89_27far8_98_89_3far8_98_89_5far8_98_89_9far_BOU_MB2300_1far_BOU_MB2300_2far_BOU_MB2300_3far_BOU_MB2300_4far_BOU_T13-2far_BOU_T13-3far_BOU_T13-5far_BOU_Tonu9far_BOU_buka-7far_BOU_buka-8far_BOU_tuna50far_BOU_tuna53far_BOU_tuna56far_BOU_tuna61far_GUA-GIL-85-5far_GUA-LUN-85-3far_GUA-MAK-85-21far_GUA-MAK-85-30far_GUA-VAS-85-2far_GUA1812_19far_GUA1815_3far_GUA1815_4far_GUA1815_8far_GUA1815_9far_GUA183_132far_GUA183_22far_GUA185_2far_GUA187_1far_GUA_HBFW16far_GUA_HBFW25far_GUA_HBFW33far_GUA_HBFW34far_GUA_HBFW4far_Man-2far_Man_Mn_1far_Man_Mn_10far_Man_Mn_2far_Man_Mn_3far_Man_Mn_4far_Man_Mn_5far_Man_Mn_6far_Man_Mn_7far_Man_Mn_8far_Man_Mn_9far_Manus-8-1far_Manus-8-2far_Manus-8-3far_Manus-8-4far_Manus-8-5far_Manus-8-6far_Manus-8-7far_Manus-8-8far_NT-PC35far_NT-cas3far_NT-cas8far_NT103-4far_NT163-2far_NT1680-1far_NT1680-3far_NT2124-2far_NT4-1far_NT_NT103-2far_NT_Nhu_10far_NT_Nhu_10afar_NT_Nhu_11afar_N

In [ ]:
pc_test